In [1]:
# default_exp utils

In [2]:
%load_ext autoreload
%autoreload 2

In [19]:
#export
from typing import List
from fastcore.basics import typed
from fastcore.test import test_eq
from toolz import thread_first, thread_last
import proseflow.text as txt

In [14]:
#export
@typed
def create_embedding_files_for_visualization(metadata, vectors, metadata_headers=None):
    """ Create embedding files for visualization. Sentences can be any kind of metadata """
    metadata = [*metadata]
    assert len(metadata) == len(vectors)

    vectors_filepath = f"/results/vectors.tsv"
    metadata_filepath = f"results/metadata.tsv"

    out_vectors = open(vectors_filepath, "w", encoding="utf-8")
    out_metadata = open(metadata_filepath, "w", encoding="utf-8")

    # Meta File Header
    if metadata_headers:
        out_metadata.write("\t".join(metadata_headers) + "\n")

    for i in range(len(vectors)):
        out_metadata.write("\t".join(metadata[i]) + "\n")
        out_vectors.write("\t".join([str(x) for x in vectors[i]]) + "\n")

    out_vectors.close()
    out_metadata.close()

In [12]:
#export
def pipe(*funcs:List[callable], thread="first"):
    thread = thread_first if thread == "first" else thread_last
    return lambda data: thread(data, *funcs)

In [16]:
for s_func in txt.STRING_FUNCS: #PYTHON MAGIC
    exec("%s=getattr(str, s_func)" %s_func)

In [21]:
clean_sentence = pipe(strip,
                      lower)

test_eq("this is a test", clean_sentence("   THIS iS a TEsT  "))